# Исследование надёжности заёмщиков

В ходе исследования определим факторы влияющие на факт погашения кредита в срок.
Ответим на следующие вопросы:

- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования необходимы для построения модели **кредитного скоринга**.

## Открытие файла с данными и изучение общей информации

In [1]:
import pandas as pd
from pymystem3 import Mystem

df = pd.read_csv('/datasets/data.csv')
#Общая информация о данных
df.info()
#Просмотр первых строк таблицы
df.head(10)
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Первичный анализ данных показал, что в таблице с данными имеются следующие проблемы:
- Пропуски в колонках 'days_employed', 'total_income';
- Вызывают вопросы данные в колонке 'days_employed'. Почему есть отрицательные значения? Почему количество дней не целочисленное? Встречаются слишком большие заначения, не соответствующие рабочему стажу ни в днях ни в часах;
- В колонке 'education' имеются одинаковые категории в разном регистре.
- В колонке 'dob_years' встречается нулевой возраст

## Предобработка данных

### Обработка пропусков

До начала работы с пропусками в количественных данных, необходимо разобраться со "странными" данными в колонке 'days_employed'. Предположим, что отрицательные значения - ошибка ввода. Поэтому просто приведм их к положительным значениям. Для решения проблемы слишком больших значений в данной колонке, необходимо дополнительно исследовать данные.

In [2]:
#В приведм данные в колонке 'days_employed' к положительным значениям
df['days_employed'] = df['days_employed'].abs()

#Выведем на экран таблицу со слишком большим стажем
df_big_employed = df[df['days_employed'] > 200000]
df_big_employed[df_big_employed['dob_years'] != 0].sort_values(by='dob_years')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1242,0,334764.259831,22,Среднее,1,Не женат / не замужем,4,F,пенсионер,0,89368.600062,получение высшего образования
16166,0,364348.197352,26,среднее,1,гражданский брак,1,M,пенсионер,0,80044.196101,автомобиль
19439,0,389397.167577,26,высшее,0,женат / замужем,0,F,пенсионер,0,214963.301941,покупка недвижимости
12753,1,329781.704997,27,среднее,1,женат / замужем,0,F,пенсионер,0,65198.102341,получение образования
13953,0,376824.585817,27,среднее,1,в разводе,3,M,пенсионер,0,97961.993557,операции с жильем
...,...,...,...,...,...,...,...,...,...,...,...,...
1826,0,368375.048770,73,Среднее,1,вдовец / вдова,2,F,пенсионер,0,74284.473064,покупка жилой недвижимости
2557,0,372861.103965,74,среднее,1,женат / замужем,0,F,пенсионер,0,42927.300898,автомобили
3460,0,344623.836105,74,среднее,1,женат / замужем,0,M,пенсионер,0,54754.745517,операции со своей недвижимостью
4895,0,341528.126150,74,высшее,0,женат / замужем,0,F,пенсионер,0,134935.354225,покупка своего жилья


Слишком большой стаж только у пенсионеров? Проверим.

In [3]:
#Выведем на экран таблицу с пенсионерами
df_pensioner = df[df['income_type'] == 'пенсионер']
df_pensioner

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,386497.714078,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,362161.054124,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


Очень похоже! Надо убедится.

In [4]:
if df_big_employed.info() == df_pensioner.info():
    print('Таблицы совпадают!')
else:
    print('Таблицы не совпадают!')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3445 entries, 4 to 21521
Data columns (total 12 columns):
children            3445 non-null int64
days_employed       3445 non-null float64
dob_years           3445 non-null int64
education           3445 non-null object
education_id        3445 non-null int64
family_status       3445 non-null object
family_status_id    3445 non-null int64
gender              3445 non-null object
income_type         3445 non-null object
debt                3445 non-null int64
total_income        3445 non-null float64
purpose             3445 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 349.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3856 entries, 4 to 21521
Data columns (total 12 columns):
children            3856 non-null int64
days_employed       3443 non-null float64
dob_years           3856 non-null int64
education           3856 non-null object
education_id        3856 non-null int64
family_status       3856 non

Если не учитывать пропуски, то таблицы полностью совпадают. Можно сделать вывод, что для пенсионеров в колонке 'days_employed' содержится информация не о трудовом стаже, а что-то другое. Например номер пенсионного удостоверения. В рамках текущего исследования, менять слишком большой стаж без точной информации о специфике этих данных считаю не целесообразным. Учтём полученные данные при заполнении пропусков.
Кроме явных 'NaN' пропусков в колонке 'dob_years' есть нулевые значения, что также является пропуском данных.
Заполнять пропуски будем следующими сособами:
- 'days_employed': средними значениями стажа в возрастной категории
- 'total_income' : медианными значениями дохода в возрастной категории 
- 'dob_years'    : на основе среднего трудового стажа.
- если пропуски есть одновременно в 'days_employed' и в 'dob_years' то ничего восстановить уже не выйдет, и эти строки придётся удалить.

In [5]:
#Удалим строки в которых есть пропуски есть одновременно в 'days_employed' и в 'dob_years' 
df = df.drop(df[(df['days_employed'].isna()) & (df['dob_years'] == 0)].index)


In [6]:
#Сисок возрастных категорий
age_category = [[19, 25],
                [25, 30],
                [31, 40],
                [41, 50],
                [51, 75]]
#Функция, которая возвращает средний стаж
def mean_employed(row):
    for cat in age_category:
        age = row['dob_years']
        if cat[0] <= age <= cat[1]:
            if row['income_type'] == 'пенсионер':
                return cat[4]
            else:
                return cat[3]
#Функция, которая возвращает медианный доход            
def median_income(row):
    for cat in age_category:
        age = row['dob_years']
        if cat[0] <= age <= cat[1]:
            return cat[2]
            
def age_from_employ(row):
    employ = row['days_employed']
    if row['income_type'] == 'пенсионер':
        pens = 4
    else:
        pens = 3
    min_comp = 900000
    for cat in age_category:
        compare = abs(cat[pens] - employ)
        if compare < min_comp:
            min_comp = compare
            result_age = int((cat[1] + cat[0])/2) 
    return result_age
        
#Определяем средний стаж и медианный доход по возрастным категориям
for i in range(len(age_category)):
    cat = age_category[i]
    df_cat = df[(df['dob_years'] >= cat[0]) & (df['dob_years'] <= cat[1])] #Фильтр по возрасту
    age_category[i].append(df_cat['total_income'].median()) #Записываем медианный доход 
    age_category[i].append(df_cat[df_cat['income_type'] != 'пенсионер']['days_employed'].mean()) #Записываем средний стаж  
    age_category[i].append(df_cat[df_cat['income_type'] == 'пенсионер']['days_employed'].mean()) #Записываем средний стаж пенсионера 

#Заполняем пропуски
df.loc[(df['dob_years' ] == 0),['dob_years' ]] = df[df['dob_years' ] == 0].apply(age_from_employ, axis=1)
df.loc[(df['days_employed'].isna()),['days_employed']] = df[df['days_employed'].isna()].apply(mean_employed, axis=1)
df.loc[(df['total_income' ].isna()),['total_income' ]] = df[df['total_income' ].isna()].apply(median_income, axis=1)


df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Пропуски заполнены. При этом удалили всего 10 строк из таблицы. Остальные проуски заолнены на основе других ячеек.

### Замена типа данных

Тип данных колонок 'days_employed' и 'total_income' - float. Однако количество дней должно быть целым числом. Размер дохода, также необходимо привести к целочисленному.

In [7]:
#Преобразование типов
df['days_employed'] = df['days_employed'].astype('int')
df['total_income' ] = df['total_income' ].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21515 entries, 0 to 21524
Data columns (total 12 columns):
children            21515 non-null int64
days_employed       21515 non-null int64
dob_years           21515 non-null int64
education           21515 non-null object
education_id        21515 non-null int64
family_status       21515 non-null object
family_status_id    21515 non-null int64
gender              21515 non-null object
income_type         21515 non-null object
debt                21515 non-null int64
total_income        21515 non-null int64
purpose             21515 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


**Вывод**

Замена типа данных повысит наглядность данных и ускорит обработку. 

### Обработка дубликатов

Сначала проверим есть ли полные дубликаты строк. При необходимости их обработаем. Затем исправим дубликаты в категориальных данных.

In [8]:
#есть ли полные дубликаты?
df.duplicated().sum()

54

Возможной причиной возникновения полных дубликатов строк таблицы, наиболее вероятно, является ошибочный повторный ввод данных.

In [9]:
#удаляем дубликаты
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

In [10]:
#проверим колонки на дубликаты
print(df['education'].value_counts())
print(df['family_status'].value_counts())
print(df['gender'].value_counts())
print(df['income_type'].value_counts())
print(df['children'].value_counts())

среднее                13698
высшее                  4708
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   267
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64
женат / замужем          12340
гражданский брак          4160
Не женат / не замужем     2809
в разводе                 1194
вдовец / вдова             958
Name: family_status, dtype: int64
F      14181
M       7279
XNA        1
Name: gender, dtype: int64
сотрудник          11086
компаньон           5078
пенсионер           3834
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64
 0     14099
 1      4808
 2      2052
 3       

Дубликаты есть в колонке 'education'. Решим эту проблему приведением всех данных в этом столбце к нижнему регистру.
В столбце 'gender' есть проблема в одной строке с недостоверно указанным полом. В столбце 'children' есть ошибки связанные, скорее всего, с не корректным вводом данных: -1 ребёнок - преобразуем в 1, 20 детей - опечатка - преобразуем в 2.

In [11]:
#приведём данные к нижнему регистру
df['education'] = df['education'].str.lower()
#проверим
df['education'].value_counts()

среднее                15181
высшее                  5248
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [12]:
#Посмотрим на лицо с неопределнным полом
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10683,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


In [13]:
#так как в таблице женщин больше в 2 раза чем мужчин, пусть это будет женщина
df.loc[df['gender'] == 'XNA', 'gender'] = 'F'
print(df['gender'].value_counts())
#Преобразуем -1 и 20 детей
df.loc[df['children'] == -1, 'children'] = 1
df.loc[df['children'] == 20, 'children'] = 2
print(df['children'].value_counts())

F    14182
M     7279
Name: gender, dtype: int64
0    14099
1     4855
2     2128
3      329
4       41
5        9
Name: children, dtype: int64


**Вывод**

Решена проблема с полными дубликатами строк и с дубликатами в разных регистрах. Также исправлены некоторые ошибки в данных.

### Лемматизация

В колонке с целями кредита, слишком много разных уникальных значений, по смыслу похожих. Необходимо категоризировать их по смыслу. Для этого применим лемматизацию.

In [14]:
#создаём список уникальных целей кредита
purpose = df['purpose'].unique()

df['purpose'].value_counts()

свадьба                                   792
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     649
покупка жилья                             646
жилье                                     644
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              631
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка своего жилья                      620
покупка недвижимости                      620
строительство недвижимости                619
ремонт жилью                              606
покупка жилой недвижимости                605
на покупку своего автомобиля              505
заняться высшим образованием      

In [15]:
m = Mystem()
#создаём таблицу: ['полная фраза', '[список лем']]
purpose_table = []
for item in purpose:
    row = [item, m.lemmatize(item)]
    purpose_table.append(row)
#чистим список лем от коротких слов и пробелов   
for i in range(len(purpose_table)):
    new_lem = []
    for item in purpose_table[i][1]:
        if len(item) > 2:
            new_lem.append(item)
    purpose_table[i][1] = new_lem
#Создаём список категорий по лемам состоящим из одного слова
category = []
for item in purpose_table:
    if (len(item[1]) == 1) & (item[1][0] not in category):             
        category.append(item[1][0])
#определяем к каким категориям относятся фразы из полного списка
for i in range(len(purpose_table)):
    for cat in category:
        if cat in purpose_table[i][1]:
            purpose_table[i].append(cat)
#Категории жильё и недвижимость даже лематизация не объединила
#Заменим жильё на недвижимость, так как это, фактически, одна категория
for i in range(len(purpose_table)):
    if purpose_table[i][2] == 'жилье':
        purpose_table[i][2] = 'недвижимость'
        
for i in purpose_table:
    print(i)


['покупка жилья', ['покупка', 'жилье'], 'недвижимость']
['приобретение автомобиля', ['приобретение', 'автомобиль'], 'автомобиль']
['дополнительное образование', ['дополнительный', 'образование'], 'образование']
['сыграть свадьбу', ['сыграть', 'свадьба'], 'свадьба']
['операции с жильем', ['операция', 'жилье'], 'недвижимость']
['образование', ['образование'], 'образование']
['на проведение свадьбы', ['проведение', 'свадьба'], 'свадьба']
['покупка жилья для семьи', ['покупка', 'жилье', 'для', 'семья'], 'недвижимость']
['покупка недвижимости', ['покупка', 'недвижимость'], 'недвижимость']
['покупка коммерческой недвижимости', ['покупка', 'коммерческий', 'недвижимость'], 'недвижимость']
['покупка жилой недвижимости', ['покупка', 'жилой', 'недвижимость'], 'недвижимость']
['строительство собственной недвижимости', ['строительство', 'собственный', 'недвижимость'], 'недвижимость']
['недвижимость', ['недвижимость'], 'недвижимость']
['строительство недвижимости', ['строительство', 'недвижимость'],

In [16]:
#Замена значений в колонке 'purpose' на лематизированные
def purpose_rename(purpose_frse):
    for purp in purpose_table:
        if purpose_frse == purp[0]:
            return purp[2]
        
df['purpose'] = df['purpose'].apply(purpose_rename)


In [17]:
df['purpose'].value_counts()

недвижимость    10806
автомобиль       4308
образование      4014
свадьба          2333
Name: purpose, dtype: int64

**Вывод**

Лематизация помогла привести три десятка целей кредитования к четырём.

### Категоризация данных

В таблице с данными можно выделить следующие категориальные данные:
- 'dob_years' - Для дальнейшего анализа данных, будет полезным возраст представить в виде возрастных категорий.
- 'gender' - занимает один столбец и содержит по одному символу в строке. Нет смысла что либо ещё с этим делать.
- 'education', 'family_status', 'income_type' и 'purpose' для удобства работы, уменьшения объёма занимаемой памяти и увеличения скорости обработки произведм категоризацию по значению. Необходимо создать словарь для каждой из этих колонок, а в основной таблице оставим только идентификаторы категорий. Причём, для двух из них, идентификаторы уже есть: 'education_id' и'family_status_id'. 
- Для дальнейшего анализа, необходимо выделить категорию наличия и отсутствия детей, а также выдилить категории по уровню общего дохода.


In [18]:
#Создание словаря для 'education'
education_dict = df[['education','education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
#Т.к. индексы строк и значения колонки 'education_id' равны, в качестве идентификаторов будем использовать индексы строк.
education_dict = education_dict.drop(columns='education_id')
display(education_dict)

#Создание словаря для 'family_status'
family_status_dict = df[['family_status','family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
#Т.к. индексы строк и значения колонки 'family_status_id' равны, в качестве идентификаторов будем использовать индексы строк.
family_status_dict = family_status_dict.drop(columns='family_status_id')
display(family_status_dict)

#Создание словаря для 'income_type'
income_type_dict = df[['income_type']]
income_type_dict = income_type_dict.drop_duplicates().reset_index(drop=True)
display(income_type_dict)

#Создание словаря для 'purpose'
purpose_dict = df[['purpose']]
purpose_dict = purpose_dict.drop_duplicates().reset_index(drop=True)
display(purpose_dict)

,education
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


,family_status
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,Не женат / не замужем


,income_type
0,сотрудник
1,пенсионер
2,компаньон
3,госслужащий
4,безработный
5,предприниматель
6,студент
7,в декрете


,purpose
0,недвижимость
1,автомобиль
2,образование
3,свадьба


In [19]:
#Добавление колонки 'income_type_id'
def id_income_type(income):
    return  income_type_dict[income_type_dict['income_type'] == income].index[0]

df['income_type_id'] = df['income_type'].apply(id_income_type)

In [20]:
#Добавление колонки 'purpose_id'
def id_purpose(purp):
    return  purpose_dict[purpose_dict['purpose'] == purp].index[0]


df['purpose_id'] = df['purpose'].apply(id_purpose)

In [21]:
#Добавление колонки возрастной категории 'years_cat'
def years_category(year):
    for cat in age_category:
        if cat[0] <= year <= cat[1]:
            return str(cat[0]) + '-' + str(cat[1])
            
df['years_cat'] = df['dob_years'].apply(years_category)

In [22]:
#Добавление колонки катенории наличие/отсутствие детей 'child_ststus'
def child_category(child):
    if child == 0:
        return 0
    else:
        return 1
            
df['child_status'] = df['children'].apply(child_category)

In [23]:
#Сисок категорий уровня доходов
total_income_cat = [100000, 150000, 200000, 300000, 2500000]

def income_category(income):
    for cat in total_income_cat:
        if income <= cat:
            return cat

df['income_category'] = df['total_income'].apply(income_category)

In [24]:
#Удаление лишних колонок
df = df.drop(columns=['education', 'family_status', 'income_type', 'purpose'])
df['income_category'].value_counts()

150000     6742
200000     5190
100000     4463
300000     3584
2500000    1482
Name: income_category, dtype: int64

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21461 entries, 0 to 21460
Data columns (total 13 columns):
children            21461 non-null int64
days_employed       21461 non-null int64
dob_years           21461 non-null int64
education_id        21461 non-null int64
family_status_id    21461 non-null int64
gender              21461 non-null object
debt                21461 non-null int64
total_income        21461 non-null int64
income_type_id      21461 non-null int64
purpose_id          21461 non-null int64
years_cat           21461 non-null object
child_status        21461 non-null int64
income_category     21461 non-null int64
dtypes: int64(11), object(2)
memory usage: 2.1+ MB


**Вывод**

Произвдена обработка категоризированных данных, а также введены новые категории. Данные меры позволят более качественно анализировать данные.

## Шаг 3. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [26]:
#Сводная таблица
children_pivot = df.pivot_table(index='child_status', columns='debt', values='purpose_id', aggfunc='count')
children_pivot['all'] = children_pivot[0] + children_pivot[1]
children_pivot['percent'] = children_pivot[1] / children_pivot['all'] * 100
children_pivot['child_status'] = ['Бездетные','С детьми']
display(children_pivot)
#Сравнение процента задолженности клиентов с детьми и бездетных
percent_cild = 100 - children_pivot.loc[0, 'percent'] / children_pivot.loc[1,'percent'] * 100
print(f'Замщики с детьми, в среднем на {percent_cild:.3}% чаще имеют задолженности чем бездетные.')

debt,0,1,all,percent,child_status
child_status,,,,,
0,13036,1063,14099,7.539542,Бездетные
1,6684,678,7362,9.209454,С детьми


Замщики с детьми, в среднем на 18.1% чаще имеют задолженности чем бездетные.


**Вывод**

Зависимость между наличием детей и возвратом кредита в срок есть. Замщики с детьми, в среднем на 18.1% чаще имеют задолженности чем бездетные.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
#Сводная таблица
family_pivot = df.pivot_table(index='family_status_id', columns='debt', values='purpose_id', aggfunc='count')
family_pivot['all'] = family_pivot[0] + family_pivot[1]
family_pivot['percent'] = family_pivot[1] / family_pivot['all'] * 100
family_pivot['family_status_id'] = family_status_dict['family_status']
display(family_pivot)
#Сравнение процента задолженности между категориями
percent_avr = 100 - (family_pivot.loc[2:3, 'percent'].mean()) / family_pivot.loc[0,'percent'] * 100
print(f'Женатые и замужние замщики, в среднем на {percent_avr:.3}% чаще имеют задолженности чем разведнные и вдовы.')
percent_worse = 100 - (family_pivot.loc[2:3, 'percent'].mean()) / family_pivot.loc[[1,4],'percent'].mean() * 100
print(f'Живущие гражданским браком и разведённые, в среднем на {percent_worse:.3}% чаще имеют задолженности чем разведнные и вдовы.')

debt,0,1,all,percent,family_status_id
family_status_id,,,,,
0,11409,931,12340,7.544571,женат / замужем
1,3772,388,4160,9.326923,гражданский брак
2,895,63,958,6.576200,вдовец / вдова
3,1109,85,1194,7.118928,в разводе
4,2535,274,2809,9.754361,Не женат / не замужем


Женатые и замужние замщики, в среднем на 9.24% чаще имеют задолженности чем разведнные и вдовы.
Живущие гражданским браком и разведённые, в среднем на 28.2% чаще имеют задолженности чем разведнные и вдовы.


**Вывод**

Зависимость между семейным положением и возвратом кредита в срок, так же, прослеживается.
- Самые наджные замщики - вдовцы, вдовы и люди в разводе. 
- Следующие по рейтингу - семейные. Они, в среднем, на 9.24% чаще имеют задолженности чем разведнные и вдовы.
- Самые не надёжные - живущие гражданским браком и разведённые. Они, в среднем, на 28.2% чаще имеют задолженности чем разведнные и вдовы.


### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
#Сводная таблица
income_pivot = df.pivot_table(index='income_category', columns='debt', values='purpose_id', aggfunc='count')
income_pivot['all'] = income_pivot[0] + income_pivot[1]
income_pivot['percent'] = income_pivot[1] / income_pivot['all'] * 100
display(income_pivot)

#Сравнение процента задолженности клиентов между категориями
poor_cry_more = 100 - (income_pivot.loc[300000:2500000, 'percent'].mean()) / income_pivot.loc[100000:200000,'percent'].mean() * 100
print(f'Люди с низким и средним уровнем дохода, в среднем на {poor_cry_more:.3}% чаще имеют задолженности чем люди с высокими доходами.')

debt,0,1,all,percent
income_category,,,,
100000,4109,354,4463,7.931884
150000,6167,575,6742,8.528627
200000,4736,454,5190,8.747592
300000,3332,252,3584,7.031250
2500000,1376,106,1482,7.152497


Люди с низким и средним уровнем дохода, в среднем на 15.6% чаще имеют задолженности чем люди с высокими доходами.


**Вывод**

Зависимость между уровнем дохода и возвратом кредита в срок, определённо есть. Люди с низким и средним уровнем дохода, в среднем на 15.6% чаще имеют задолженности чем люди с высокими доходами.

### Как разные цели кредита влияют на его возврат в срок?

In [29]:
#Сводная таблица
purpose_pivot = df.pivot_table(index='purpose_id', columns='debt', values='total_income', aggfunc='count')
purpose_pivot['all'] = purpose_pivot[0] + purpose_pivot[1]
purpose_pivot['percent'] = purpose_pivot[1] / purpose_pivot['all'] * 100
purpose_pivot['family_status_id'] = purpose_dict['purpose']
display(purpose_pivot)

#Сравнение процента задолженности между категориями
percent_auto_obr = 100 - (purpose_pivot.loc[0, 'percent']) / purpose_pivot.loc[1:2,'percent'].mean() * 100
print(f'Берущие кредит на автомобиль или образование, в среднем на {percent_auto_obr:.3}% чаще имеют задолженности чем берущие кредит на недвижимость')
percent_marry = 100 - (purpose_pivot.loc[0, 'percent']) / purpose_pivot.loc[3,'percent'] * 100
print(f'Берущие кредит на свадьбу, в среднем на {percent_marry:.3}% чаще имеют задолженности чем берущие кредит на недвижимость')


debt,0,1,all,percent,family_status_id
purpose_id,,,,,
0,10024,782,10806,7.236720,недвижимость
1,3905,403,4308,9.354689,автомобиль
2,3644,370,4014,9.217738,образование
3,2147,186,2333,7.972568,свадьба


Берущие кредит на автомобиль или образование, в среднем на 22.1% чаще имеют задолженности чем берущие кредит на недвижимость
Берущие кредит на свадьбу, в среднем на 9.23% чаще имеют задолженности чем берущие кредит на недвижимость


**Вывод**

Цели кредита влияют на его возврат в срок.
- Самые надёжные замщики - люди берущие кредит на недвижимость.
- Люди берущие кредит на свадьбу, в среднем на 9.23% чаще имеют задолженности чем берущие кредит на недвижимость
- Люди берущие кредит на автомобиль или образование, в среднем на 22.1% чаще имеют задолженности чем берущие кредит на недвижимость

## Общий вывод

В результате исследования можно сделать следующие выводы:
 - Существует зависимость между наличием детей и возвратом кредита в срок. Замщики с детьми, в среднем на 18% чаще имеют задолженности чем бездетные.
 - Существует и зависимость между семейным положением и возвратом кредита в срок. Самые наджные замщики - вдовцы, вдовы и люди в разводе. Следующие по рейтингу - семейные. Самые не надёжные - живущие гражданским браком и разведённые.
 - Прослеживается и зависимость между уровнем дохода и возвратом кредита в срок. Люди с низким и средним уровнем дохода, в среднем на 15.6% чаще имеют задолженности чем люди с высокими доходами.
 - Цели кредита тоже влияют на его возврат в срок. Самые наджные заёмщики - берущие кредит на недвижимость.
 
Портрет идеального заёмщика: Бездетный, разведённый или вдовец, с высоким доходом, берущий кредит на недвижимость. 